In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import cftime
from datetime import date


In [ ]:
filePath = '/glade/collections/cdg/data/cordex/data/raw/NAM-22i/day/RegCM4/HadGEM2-ES/hist/tasmax/tasmax.hist.HadGEM2-ES.RegCM4.day.NAM-22i.raw.nc'
ds = xr.open_dataset(filePath)
ds

In [ ]:
## Create a small slice with the full time axis for local testing. 

#subset = ds.isel(lat=0, lon=0)
#subset
#subset.to_netcdf('/glade/u/home/bonnland/cal360_data.nc')

In [ ]:
def get_datemap_360_to_noleap():
    ''' Return an array of dates mapping days from the 360-Day calendar to the No-Leap calendar. '''

    # Choose any year with 365 days. 
    dummy_year = 1999

    # These are the days of the year that will be missing on the time axis for each year. 
    # The goal is to spread missing dates out evenly over each year.
    #
    # Modify specific dates as desired. 
    missing_dates = [date(dummy_year, 1, 31),
                     date(dummy_year, 3, 31),
                     date(dummy_year, 5, 31),
                     date(dummy_year, 8, 31),
                     date(dummy_year, 10, 31),]
    
    day_one = date(dummy_year, 1, 1)
    missing_dates_indexes = [(day - day_one).days for day in missing_dates]    

    datemap_indexes = np.setdiff1d(np.arange(365), missing_dates_indexes)
    
    dates = pd.date_range(f'1/1/{dummy_year}', f'12/31/{dummy_year}')
    assert(len(dates) == 365)
    
    date_map = dates[datemap_indexes]
    assert(len(date_map) == 360)
    return date_map


def convert_to_noleap(cftime360, datemap):
    newdate = datemap[cftime360.dayofyr - 1]
    converted = cftime.DatetimeNoLeap(year=cftime360.year, month=newdate.month, day=newdate.day)
    return converted

In [ ]:
datemap = get_datemap_360_to_noleap()
datemap[0]

In [ ]:
ds['time'] = [convert_to_noleap(t, datemap) for t in ds.time.values]
ds

In [ ]:
ds['time'][:60]

In [ ]:
ds["time"] = ds.indexes["time"].to_datetimeindex()
ds['time'][:60]